In [1]:

!pip install pyautogen[retrievechat]==0.2.3
!pip install amazon-codewhisperer-jupyterlab-ext

     -------------------------------------- 133.9/133.9 kB 2.0 MB/s eta 0:00:00
     -------------------------------------- 381.9/381.9 kB 2.6 MB/s eta 0:00:00
     -------------------------------------- 222.3/222.3 kB 2.7 MB/s eta 0:00:00
  Using cached FLAML-2.1.1-py3-none-any.whl (295 kB)
     -------------------------------------- 786.3/786.3 kB 2.1 MB/s eta 0:00:00
  Using cached diskcache-5.6.3-py3-none-any.whl (45 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached sentence-transformers-2.2.2.tar.gz (85 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 283.9/283.9 kB 2.5 MB/s eta 0:00:00
     -------------------------------------- 509.0/509.0 kB 2.7 MB/s eta 0:00:00
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
     ---------------------------------------- 75.9/75.9 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 85.5/85.5 kB 

  DEPRECATION: sentence-transformers is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559

[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 120.8/120.8 kB 2.4 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: still running...
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 11.9/11.9 MB 2.8 MB/s eta 0:00:00
  Using cached jupyterlab-4.0.11-py3-none-any.whl (9.2 MB)
     ---------------------------------------- 76.1/76.1 kB 2.1 MB/s eta 0:00:00
     -------------------------------------- 139.3/139.3 kB 2.7 MB/s eta 0:00:00
  Using cached jupyter_server-2.12.5-py3-none-any.whl (380 kB)
  Using cached async_lru-2.0.4-py3-none-any.whl (6.1 kB)
  Using cached notebook_shim-0.2.3-py3-none-any.whl (13 kB)
  Using cached jupyterlab_server-2.25.2-py3-none-any.whl (58 


[notice] A new release of pip available: 22.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import logging
import os

from autogen import config_list_from_dotenv
from autogen.agentchat.contrib.gpt_assistant_agent import GPTAssistantAgent
from autogen import UserProxyAgent, AssistantAgent, GroupChat, GroupChatManager


In [3]:
config_list_default = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-4",
                 "gpt-3.5-turbo"}
    }
)
config_list_cheap = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-3.5-turbo"}
    }
)
config_list_costly = config_list_from_dotenv(
    dotenv_file_path=".env",
    filter_dict={
        "model":{"gpt-4"}
    }
)


# 1. First attemp: Product Manager and Developer on Simple Coding Problem

Build a team consisting of a Product Manager and a Developer, to work on a simple coding task: write Python code that generates random number.

It was interesting to see the two agents' conversation drifted away from the original problem of generating random number to the ambitious project of developing a software product that uses Machine Learning to analyse the emotions from a user's text input in real time.

In [23]:


llm_config = {"config_list": config_list_cheap,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}


user_proxy = UserProxyAgent(name="user_proxy",
                            system_message="A human adimin.",
                            max_consecutive_auto_reply=10,
                            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config={
                                "work_dir": "coding",
                                "use_docker":False,
                            },
                            human_input_mode="NEVER")

pm = AssistantAgent(name="Product_manager",
                    system_message="Creative in software product ideas. Don't write any code. Just lay out the plan for what the"
                                    "coder need to code.",
                    llm_config=llm_config)


coder = AssistantAgent(
    name="coder",
    system_message=f"I am a 10x engineer, trained in Python. I was the first engineer at Uber" 
                   f"(which I make sure to tell everyone I work with).\n"
                   f"You are participating in a group chat with a user ({user_proxy.name})"
                   f"and a product manager ({pm.name}).",
    llm_config=llm_config
                   
)

groupchat = GroupChat(agents=[user_proxy, pm, coder], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

user_proxy.initiate_chat(manager, 
                         message="Can you create Python code that generates a random number?")

user_proxy (to manager):

Can you create Python code that generates a random number?

--------------------------------------------------------------------------------
coder (to manager):

Sure, here is a simple way to generate a random number in Python using the `random` library:

```python
import random

def generate_random_num():
  random_num = random.randint(0, 100) # generates a random number between 0 and 100
  return random_num

print(generate_random_num())
```

This code defines a function `generate_random_num()` that generates a random number between 0 and 100 (you can adjust this range to whatever you need) and then prints that number.

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


user_proxy (to manager):

exitcode: 0 (execution succeeded)
Code output: 
90


--------------------------------------------------------------------------------
user_proxy (to manager):



--------------------------------------------------------------------------------
Product_manager (to manager):

Plan:
1. Define a function `generate_primes` which will take an integer `n` as an input argument. The function will generate all prime numbers up to `n`.
    - In the function, create a list `primes` to hold the generated prime numbers.
    - Implement a nested loop where the outer loop goes through each number from `2` up to `n` (inclusive), and the inner loop goes through each number in `primes`.
    - In the inner loop, check if the current number from the outer loop is divisible by the current number from `primes`. If so, break the loop; it's not a prime number. If not, it is a prime number, so append it to `primes`.
2. Define a function `generate_random_prime` which will take an integer

user_proxy (to manager):

exitcode: 0 (execution succeeded)
Code output: 
31


--------------------------------------------------------------------------------
user_proxy (to manager):



--------------------------------------------------------------------------------
Product_manager (to manager):

Plan:
- Our software will be a web-based language learning application that helps people learn new languages by immersing them in real-world scenarios.
- The application will include three main components:
    
	1. Frontend: This will be the user interface of the application where the users will interact. It will include the following screens:
	
		- Registration/Login: Users will be able to create a new account or log into an existing one.
		- Dashboard: Once logged in, users will see an overview of their progress, upcoming lessons, etc.
		- Lesson: This is where the actual learning takes place. Each lesson will have a real-world scenario like ordering food at a restaurant, asking for direct

In [25]:
user_proxy.chat_messages

defaultdict(list,
            {<autogen.agentchat.groupchat.GroupChatManager at 0x17a1ef8f0d0>: [{'content': 'Can you create Python code that generates a random number?',
               'role': 'assistant'},
              {'content': 'Sure, here is a simple way to generate a random number in Python using the `random` library:\n\n```python\nimport random\n\ndef generate_random_num():\n  random_num = random.randint(0, 100) # generates a random number between 0 and 100\n  return random_num\n\nprint(generate_random_num())\n```\n\nThis code defines a function `generate_random_num()` that generates a random number between 0 and 100 (you can adjust this range to whatever you need) and then prints that number.',
               'name': 'coder',
               'role': 'user'},
              {'content': 'exitcode: 0 (execution succeeded)\nCode output: \n90\n',
               'role': 'assistant'},
              {'content': '', 'role': 'assistant'},
              {'content': "Plan:\n1. Define a fun

# 2. More Complex Task: Retrieve and Analyse RBA (Reserve Bank of Australia) meeting minutes

The team of agents are given the link to RBA Monetary Policy meeting minutes, and are expected to predict whether interest rate will be raised in the next month or not.

The task requries both expertises in coding/data processing and in domain of economics.

A more diversed team is built:
- Planner
- Software Engineer
- Economist
- Code Executor 
- Critic

Interaction among the agents is interesting.  

For instance, during the conversation, Critic pointed out that the document is too long to fit 2000 token limit and told Sofeware Engineer to cut document into smaller pieces.


In [12]:
llm_config = {"config_list": config_list_default,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}


user_proxy = UserProxyAgent(name="user_proxy",
                            system_message='''A human admin. Interact with the planner to discuss the plan. 
                            Plan execution needs to be approved by this admin.''',
                            max_consecutive_auto_reply=10,
                            is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
                            code_execution_config=False,
                            #human_input_mode="NEVER",
                            )

planner = AssistantAgent(name="Planner",
                    system_message='''Planner. Suggest a plan. 
                                        Revise the plan based on feedback from admin and critic, until admin approval.
                                        The plan may involve an engineer who can write code, an executor who can execute the code to get result, and a economist who analyses the result.
                                        A critic is also a member of the team who can provide feedback to plan, code, result of execution, and analysis by economist. 
                                        Explain the plan first. 
                                        Be clear which step is performed by which memeber. All members can proivde comment and opinion. 
                                        
''',
                    llm_config=llm_config)


engineer = AssistantAgent(
    name="Engineer",
    system_message=f"You are a 10x engineer, trained in Python. You analyse requirement, and write Python script. If the requirement is not clear, you ask for clarification."
                    f"If the requirement involves scraping web pages, your code saves scrapped text to one or multiple local files locally, each file is not over 2000 token limit." 
                   f"You don't execute code. You give your script to an exectuor to execute.\n"
                   f"If the executor reports error or failure of execution, you debug and fix your script."
                   f"If you find the requirement is not achiveable, let planner and other members know.",

    llm_config=llm_config
                   
)

critic = AssistantAgent(
    name="Critic",
    system_message=f"Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. "
                    f"You don't write or execute code, don't analyse document directly. You provide feedback to members specialized at these tasks.",
    llm_config=llm_config,
)

executor = UserProxyAgent(
    name="Executor",
    system_message=f"Executor. Execute the code written by the engineer, save the result in one or multiple files locally."
                    f"If execution succeeds, you read out the texts in saved files to other members. If multiple files are saved, you read them one by one."
                    f"If execution fails, or result is empty, make sure to raise the issue to planner or engineer.",
    human_input_mode="NEVER",
    code_execution_config={"last_n_messages": 3, "work_dir": "work_dir"},
)

economist = AssistantAgent(
    name="economist",
    system_message=f"Economist. Provide economic insights. "
                    f"You read and analyse one or multiple texts provided by executor, and provide your professional insights in reponse to user question."
                    f"Your analysis must be supported by the provided texts and your knowledge. You are forbidden to make up texts or fake analysis.",  
    llm_config=llm_config,
)

groupchat = GroupChat(agents=[user_proxy, planner, engineer, critic, executor, economist], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

user_proxy.initiate_chat(manager, 
                         message="Can you analyse the material at link https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html, and tell me whether the Australian Reserve Bank will increase interest rate in next Month or not?")

user_proxy (to manager):

Can you analyse the material at link https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html, and tell me whether the Australian Reserve Bank will increase interest rate in next Month or not?

--------------------------------------------------------------------------------
Planner (to manager):

Plan:
1. Engineer: The engineer will start by writing a code to scrape the textual data from the provided URL. To accomplish this, they may use various programming languages like Python with libraries such as Beautiful Soup for web scraping and Requests for accessing the URL.

2. Executor: The executor will run the scraping code written by the engineer. The output of this step should be the raw textual data from the given URL.

3. Economist: This textual data will be reviewed by the economist, who will analyze the content to understand the monetary policy discussed in the document, specifically looking for any indications or trends that suggest th

Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...


Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------
Executor (to manager):



--------------------------------------------------------------------------------
Critic (to manager):

Looking at the code provided by the Engineer, there is little for the Critic to specifically analyze as the code is straightforward with defined tasks. The Python script is used to extract the text from the web page using BeautifulSoup and the requests library. The text is then stored in a .txt file for review. 

However, I would propose a minor alteration. Currently, only the first 2000 tokens of the extracted text are saved to the 'rba_minutes.txt'. Given that we cannot definitively know the structure of the webpage and where the relevant information may be located, I would suggest that all scraped text be saved instead of limiting it to the first 2000 tokens. This will ensure no potentially crucial infor

Executor (to manager):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 12, in <module>
    text = scrape_website(url)
NameError: name 'scrape_website' is not defined


--------------------------------------------------------------------------------
Engineer (to manager):

I apologize for the confusion. The modified script failed to execute properly due to a NameError. The 'scrape_website' function was not included in the presented script. I will amend this and present the entire script including the function definition.

```python
import requests
from bs4 import BeautifulSoup

def scrape_website(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup.get_text()

def save_to_file(data, base_filename):
    chars_per_file = 2000
    # split the data into chunks of 2000 chars
    chunks = [data[i:i + chars_per_file] for i in range(0, len(data), chars_per_file)]
    
    for i, chunk in en

Executor (to manager):

exitcode: 0 (execution succeeded)
Code output: 


--------------------------------------------------------------------------------


<B> The document is too long for the economist to read due to token number limit of LLM. 

The solution would be store long documents into vector store. When the economist carries out analysis, it retrieves relevant paragraphs which are small enough to fit in inputs. 



## Use Retrieval Agents 

Autogen implemented the RAG functionality using Retrival Agents.

Retrieval Agents store documents into a vector store, and retrieve relevant pieces when needed.



In [5]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import chromadb

In [6]:
llm_config = {"config_list": config_list_default,
                "seed": int(os.getenv("seed")),
                "timeout": int(os.getenv("request_timeout"))}

termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

user_proxy = UserProxyAgent(name="user_proxy",
                            system_message='''A human admin. Interact with the planner to discuss the plan. 
                            Plan execution needs to be approved by this admin.''',
                            max_consecutive_auto_reply=10,
                            is_termination_msg=termination_msg,
                            code_execution_config=False,
                            human_input_mode="TERMINATE",
                            )

planner = AssistantAgent(name="Planner",
                    system_message='''Planner. Suggest a plan. 
                                        Revise the plan based on feedback from admin and critic, until admin approval.
                                        The plan may involve a document management specialist who retrieves relevant documents, a economist who analyses the result, 
                                        and a critic who provides feedback to planner's plan, documents retrieved by document management specialist, and analysis produced by economist. 
                                        Explain the plan first. 
                                        Be clear which step is performed by which memeber. All members can proivde comment and opinion. 
                                        ''',
                    llm_config=llm_config)

document_maagement_specialist = RetrieveUserProxyAgent(
    name="document_management_specialist",
    system_message="Document Management Specialist. Retrieve relevant documents from the internet.",
    is_termination_msg=termination_msg,
    retrieve_config={
        "task": "qa",
        "docs_path": "data/rba_minutes.txt", #"https://www.rba.gov.au/monetary-policy/rba-board-minutes/2023/2023-11-07.html",
        "chunk_token_size": 1000,
        "client": chromadb.PersistentClient(path="/tmp/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,
    human_input_mode="NEVER",
)


critic = AssistantAgent(
    name="Critic",
    system_message=f"Critic. Double check plan, claims, code from other agents and provide feedback. Check whether the plan includes adding verifiable info such as source URL. "
                    f"You don't write or execute code, don't analyse document directly. You provide feedback to members specialized at these tasks."
                    f"If you think the result is satisfactory, you announce success and terminate the task by saying 'TERMINATE'"
                    f"If you think the task is not able to be accomplished and not worth to continue, you announce failure and termianate the task by saying 'TERMINATE'",
    llm_config=llm_config,
)


economist = RetrieveAssistantAgent(
    name="economist",
    system_message=f"Economist. Provide economic insights. "
                    f"You read and analyse one or multiple texts provided by executor, and provide your professional insights in reponse to user question."
                    f"Your analysis must be supported by the provided texts and your knowledge. You are forbidden to make up texts or fake analysis.",  
    llm_config=llm_config,
)



def retrieve_content(message, n_results=3):
        document_maagement_specialist.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = document_maagement_specialist._check_update_context(message)
        if (update_context_case1 or update_context_case2) and document_maagement_specialist.update_context:
            document_maagement_specialist.problem = message if not hasattr(document_maagement_specialist, "problem") else document_maagement_specialist.problem
            _, ret_msg = document_maagement_specialist._generate_retrieve_user_reply(message)
        else:
            ret_msg = document_maagement_specialist.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message


for agent in [user_proxy, planner, economist, critic]:
    # register functions for all agents.
    agent.register_function(
        function_map={
            "retrieve_content": retrieve_content,
        }
    )

groupchat = GroupChat(agents=[user_proxy, planner, document_maagement_specialist, critic, economist], 
                      messages=[], 
                      max_round=12)
manager = GroupChatManager(name="manager", 
                           groupchat=groupchat, 
                           llm_config=llm_config)

# user_proxy.initiate_chat(manager, 
#                          message="Whether Australian Reserve Bank will increase interest rate in next Month or not?")
document_maagement_specialist.initiate_chat(manager, 
                         problem="Whether Australian Reserve Bank will increase interest rate in next Month or not?")

Trying to create collection.


d:\Documents\work\vscode_workspace\mas_econo_squad\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:autogen.retrieve_utils:Found 6 chunks.


doc_ids:  [['doc_4', 'doc_5', 'doc_3', 'doc_0', 'doc_1', 'doc_2']]
Adding doc_id doc_4 to context.
Adding doc_id doc_5 to context.
Adding doc_id doc_3 to context.
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Adding doc_id doc_2 to context.
document_management_specialist (to manager):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: Whether Australian Reserve Bank will increase interest rate in next Month or not?

Context is: 
Members considered the implications for monetary policy of developments in financial conditions. They noted that the cash rate remained below policy rates in many other countries, despite similar economic conditions, although various factors might account for this difference. The rise in